In [1]:

from matplotlib import pyplot as plt
import matplotlib
import numpy as np
from PIL import Image

In [3]:

#seq = iaa.Sequential([
#    iaa.Flipud(0.6),  # Flip vertically with 60% probability
#    iaa.Affine(rotate=(-20, 15)),  # Rotate image by -20 to 15 degrees
#    iaa.GaussianBlur(sigma=(0, 2.5)),  # Apply Gaussian blur with a sigma between 0 and 2.5
#    iaa.AdditiveGaussianNoise(scale=(0, 0.1*255)),  # Add Gaussian noise with a mean of 0 and a standard deviation between 0 and 0.1*255
#    iaa.Crop(px=(0, 16)),  # Crop images from each side by 0 to 16 pixels
#])


#input_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/whiterook"
#output_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/whiterook"

# Loop through images in input directory
#for filename in os.listdir(input_dir):
#    if filename.endswith(".jpeg"):  # Only process jpg files
#        # Load image
#        image = cv2.imread(os.path.join(input_dir, filename))
        
#        # Generate augmented images
#        images_aug = seq.augment_images([image])
        
#        # Save augmented images to output directory
#        for i, image_aug in enumerate(images_aug):
#            cv2.imwrite(os.path.join(output_dir, f"{filename}_{i}aug2.jpeg"), image_aug)


In [2]:
import os
import shutil

# Define the source and destination folders
src_folder = '/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/train_data'
dst_folder = '/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/validation_data'

# Define the list of subfolders to process
subfolders = ['whitepawn', 'whitequeen', 'whiterook', 'whiteknight', 'whitebishop', 'whiteking',
              'blackpawn', 'blackqueen', 'blackrook', 'blackknight', 'blackbishop', 'blackking']

# Loop through each subfolder in the source folder
for subfolder in subfolders:
    # Get the path to the source and destination subfolders
    src_subfolder = os.path.join(src_folder, subfolder)
    dst_subfolder = os.path.join(dst_folder, subfolder)

    # Get the list of files in the subfolder
    files = os.listdir(src_subfolder)

    # Calculate the number of files to move (half of the total)
    num_files = len(files)
    num_to_move = num_files // 2

    # Create a new folder in the destination folder with the same name
    os.makedirs(dst_subfolder, exist_ok=True)

    # Move the first half of the files to the new folder
    for filename in files[:num_to_move]:
        src_file_path = os.path.join(src_subfolder, filename)
        dst_file_path = os.path.join(dst_subfolder, filename)
        shutil.move(src_file_path, dst_file_path)


In [4]:

#src_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data"
#test_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/test_data"

#test_pct = 0.3

# Loop over each subdirectory in the source directory
#for subdir in os.listdir(src_dir):
#    subdir_path = os.path.join(src_dir, subdir)
#    if os.path.isdir(subdir_path):
#        # Get a list of all files in the subdirectory
#        files = os.listdir(subdir_path)

 #       # Determine the number of files to move to the test set
 #       num_files = len(files)
 #       num_test_files = int(test_pct * num_files)

 #       # Randomly select the files to move to the test set
 #       test_files = random.sample(files, num_test_files)

 #       # Move the test files to the destination directory
  #      for filename in test_files:
  #          src_file = os.path.join(subdir_path, filename)
  #          dst_file = os.path.join(test_dir, filename)
  #          shutil.move(src_file, dst_file)


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = (480, 640)
batch_size = 128

train_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/train_data"
test_dir = "/Users/zuzannaszubartowska/code/zuzannaszu/chess_hacker/raw_data/new_chess_data/test_data"

num_classes = 12

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical')

from tensorflow.keras import layers, models

height = 480
width = 640
channels = 3
num_classes = 12

model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(height, width, channels)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))





In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 478, 638, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 239, 319, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 237, 317, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 118, 158, 64)     0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1193216)           0         
                                                                 
 dense (Dense)               (None, 32)                3

In [8]:
from keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
    monitor='val_loss', 
    min_delta=0.001, 
    patience=3, 
    verbose=1, 
    mode='auto'
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size,
    callbacks=[earlystop_callback]
)



Epoch 1/10


2023-03-02 11:06:11.490801: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


: 

: 